In [1]:
import pandas as pd

from ipywidgets import interact, Dropdown

from src.data_load import load_tables, load_online_instance, load_distances
from src.filtering import flexible_filter
from src.plotting import prepare_labor_data, plot_labors, make_histograms
from src.experimentation_config import *
from src.config import *

data_path = '../data'
instance = 'instAD1'
# instance = 'instRD1'
dist_method = 'osrm'

directorio_df, labors_raw_df, cities_df, duraciones_df, valid_cities = load_tables(data_path, generate_labors=False)
labors_real_df, labors_static_df, labors_dynamic_df = load_online_instance(data_path, instance, labors_raw_df)
dist_dict = load_distances(data_path, dist_method, instance)

fechas = fechas_dict[instance]

# Visual exploration

In [2]:
all_labors_df = prepare_labor_data(labors_static_df, labors_dynamic_df)

def create_interactive_plot(all_labors_df):
    """
    Create a dropdown widget to select date and display the plot interactively.
    """
    # Prepare unique dates as strings
    unique_dates = sorted(all_labors_df["schedule_date"].dt.normalize().unique())
    date_options = [d.strftime("%Y-%m-%d") for d in unique_dates]
    date_options.insert(0, "ALL")  # Add 'ALL' option at the top

    interact(
        lambda selected_date_str: plot_labors(all_labors_df, selected_date_str),
        selected_date_str=Dropdown(options=date_options, description="Select Date:")
    )

# Create interactive plot
create_interactive_plot(all_labors_df)


interactive(children=(Dropdown(description='Select Date:', options=('ALL', '2026-01-05', '2026-01-06', '2026-0…

In [3]:
def plot_order_timing_histograms_interactive(
    df,
    created_col='created_at',
    schedule_col='schedule_date',
    city_col='city',
    unit='hours'
):
    """
    Interactive version of order timing histograms with a city dropdown.
    """

    # --- Prepare city options ---
    if city_col not in df.columns:
        raise ValueError(f"'{city_col}' column not found in dataframe.")

    cities = sorted(df[city_col].dropna().unique().tolist())
    cities.insert(0, "ALL")

    # --- Define interactive plotting function ---
    def _plot_for_city(selected_city):
        if selected_city == "ALL":
            filtered_df = df
        else:
            filtered_df = df[df[city_col] == selected_city]

        if filtered_df.empty:
            print(f"No data available for city: {selected_city}")
            return

        fig1, fig2 = make_histograms(filtered_df, created_col, schedule_col, unit)
        fig1.show()
        fig2.show()

    # --- Create dropdown ---
    interact(
        _plot_for_city,
        selected_city=Dropdown(options=cities, description="Select City:")
    )

plot_order_timing_histograms_interactive(labors_dynamic_df)

interactive(children=(Dropdown(description='Select City:', options=('ALL', '1', '1004', '126', '149', '150', '…

# Atypic cases

## Schedule date is before Created at 

In [8]:
rel_cols = ['schedule_date', 'created_at']
labors_dynamic_df[labors_dynamic_df['schedule_date'] < labors_dynamic_df['created_at']][rel_cols+labors_dynamic_df.columns.to_list()]

,schedule_date,created_at,service_id,labor_id,labor_type,labor_name,labor_category,labor_price,labor_created_at,labor_start_date,...,paying_customer,state_service,start_address_id,start_address_point,end_address_id,end_address_point,city,address_id,address_point,address_name
16,2026-01-09 16:00:00-05:00,2026-01-09 17:17:24.432000-05:00,217887,311037.0,2.0,Alfred Transport,VEHICLE_TRANSPORTATION,55663.0,2025-03-03 17:17:24.491000-05:00,2026-01-09 16:00:00-05:00,...,20247.0,COMPLETED,3128.0,POINT (-75.5814053 6.214877200000001),3128.0,POINT (-75.5814053 6.214877200000001),1,55748.0,POINT (-75.55748439999999 6.252939599999999),Apartamento
58,2026-01-06 11:00:00-05:00,2026-01-06 11:09:13.938000-05:00,243406,340977.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,90004.0,2025-05-23 11:02:26.996000-05:00,2026-01-06 11:36:00-05:00,...,45184.0,COMPLETED,69791.0,POINT (-76.38704109999999 3.5356493),69791.0,POINT (-76.38704109999999 3.5356493),1004,19281.0,POINT (-76.5468388 3.4090961),Apartamento
152,2026-01-07 17:00:00-05:00,2026-01-07 17:03:52.648000-05:00,202911,294635.0,6.0,General Mechanics,GENERAL_MECHANICS,993650.0,2025-01-15 17:03:52.673000-05:00,2026-01-07 08:31:00-05:00,...,45184.0,COMPLETED,87960.0,POINT (-74.776479 10.8865373),87960.0,POINT (-74.776479 10.8865373),126,121049.0,POINT (-74.8051698 10.991633),Fleetser - Barranquilla
153,2026-01-07 17:00:00-05:00,2026-01-07 17:03:52.648000-05:00,202911,294996.0,2.0,Alfred Transport,VEHICLE_TRANSPORTATION,65004.0,2025-01-16 14:49:04.938000-05:00,2026-01-07 16:04:00-05:00,...,45184.0,COMPLETED,87960.0,POINT (-74.776479 10.8865373),87960.0,POINT (-74.776479 10.8865373),126,56474.0,POINT (-74.79419462825471 10.906019892143393),Casa
243,2026-01-08 17:50:00-05:00,2026-01-08 17:58:43.431000-05:00,255033,351199.0,2.0,Alfred Transport,VEHICLE_TRANSPORTATION,57789.0,2025-06-13 17:58:43.454000-05:00,2026-01-08 17:50:00-05:00,...,61607.0,COMPLETED,78914.0,POINT (-74.0459602 4.7526023),78914.0,POINT (-74.0459602 4.7526023),149,128219.0,POINT (-74.19197489999999 4.6331451),Casa
296,2026-01-05 08:30:00-05:00,2026-01-05 08:43:26.604000-05:00,211851,304331.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,57789.0,2025-02-12 08:43:26.661000-05:00,2026-01-05 08:30:00-05:00,...,61607.0,COMPLETED,123972.0,POINT (-74.0920917 4.6217267),123972.0,POINT (-74.0920917 4.6217267),149,9575.0,POINT (-74.1474839 4.6118768),Apartamento
314,2026-01-10 16:00:00-05:00,2026-01-10 16:00:02.159000-05:00,225154,318990.0,10.0,Paint Job,PAINT_JOB,4938500.0,2025-03-26 16:00:02.183000-05:00,2026-01-10 10:55:00-05:00,...,45184.0,COMPLETED,115083.0,POINT (-74.136066 4.6926739),115083.0,POINT (-74.136066 4.6926739),149,56315.0,POINT (-74.07426269999999 4.651892),Autodealer
391,2026-01-09 11:00:00-05:00,2026-01-09 11:03:22.333000-05:00,211471,303926.0,6.0,General Mechanics,GENERAL_MECHANICS,1570800.0,2025-02-11 11:03:22.359000-05:00,2026-01-09 11:11:00-05:00,...,45184.0,COMPLETED,91510.0,POINT (-75.516454 10.446315),91510.0,POINT (-75.516454 10.446315),150,94075.0,POINT (-75.5308586 10.4214759),Reyautos Del Caribe Cartagena
403,2026-01-05 14:00:00-05:00,2026-01-05 14:25:47.074000-05:00,238439,334177.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,65004.0,2025-05-08 08:11:56.308000-05:00,2026-01-05 09:40:00-05:00,...,45184.0,COMPLETED,94075.0,POINT (-75.5308586 10.4214759),91510.0,POINT (-75.516454 10.446315),150,123522.0,POINT (-75.5051041 10.392139),Apartamento
408,2026-01-05 15:00:00-05:00,2026-01-05 15:22:51.105000-05:00,239348,334362.0,2.0,Alfred Transport,VEHICLE_TRANSPORTATION,49485.0,2025-05-08 15:22:51.145000-05:00,2026-01-05 15:00:00-05:00,...,50217.0,COMPLETED,79748.0,POINT (-75.51627789999999 10.4094262),79748.0,POINT (-75.51627789999999 10.4094262),150,123522.0,POINT (-75.5051041 10.392139),Apartamento


In [1]:
x = '029748_free'
x[-4:]

'free'